In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader, TensorDataset,random_split
import matplotlib.pyplot as plt
import random
import os
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from scipy.stats import norm
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")

#Checking GPU Availability

In [2]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


#Matlab Integration

In [3]:
import matlab.engine

# Start MATLAB
eng = matlab.engine.start_matlab()

# Get the MATLAB version
version = eng.version()
print(f"Running MATLAB Version: {version}")

# Quit MATLAB
eng.quit()

Running MATLAB Version: 24.2.0.2863752 (R2024b) Update 5


#Initial Data-set

In [20]:
X=np.loadtxt('pcb_ini_200.txt')
Y=np.loadtxt('y_ini_200.txt')
x_test=np.loadtxt('pcb_ini_MO_test.txt')
S_test=np.loadtxt('S11_MO_test.txt')
G_test=np.loadtxt('Gain_MO_test.txt')
y_test = np.concatenate((S_test,G_test), axis=1)
print(X.shape,Y.shape,x_test.shape,y_test.shape)

(200, 56) (200, 200) (50, 56) (50, 200)


In [21]:
# Assuming X and Y are already defined
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

# Convert data to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
print(X_train.shape,y_train.shape,X_val.shape,y_val.shape)

torch.Size([180, 56]) torch.Size([180, 200]) torch.Size([20, 56]) torch.Size([20, 200])


#Surrogate-Model Architecture

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BayesianRegressionNet(nn.Module):
    def __init__(self, input_dim=56, output_dim=200, hidden_dims=[913, 546,1284,1897,1259,302], drop_rate=0.2):
        super().__init__()
        self.input_bn = nn.BatchNorm1d(input_dim)
        
        # Dynamically create hidden layers
        self.hidden_layers = nn.ModuleList()
        prev_dim = input_dim
        for dim in hidden_dims:
            self.hidden_layers.append(nn.Linear(prev_dim, dim))
            self.hidden_layers.append(nn.BatchNorm1d(dim))
            self.hidden_layers.append(nn.Dropout(drop_rate))
            prev_dim = dim
        
        # Output heads
        self.s11_head = nn.Linear(prev_dim, 100)
        self.gain_head = nn.Linear(prev_dim, 100)

    def forward(self, x):
        x = self.input_bn(x)
        for layer in self.hidden_layers:
            if isinstance(layer, nn.Linear):
                x = F.selu(layer(x))
            else:
                x = layer(x)
        s11 = self.s11_head(x)
        gain = self.gain_head(x)
        return torch.cat([s11, gain], dim=1)
    def init_weights(self):
        """Initialize weights using Xavier initialization."""
        for layer in self.modules():
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                nn.init.zeros_(layer.bias)
  
# Example usage
model = BayesianRegressionNet()
# print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

In [27]:
def init_weights(self):
        """Initialize weights using Xavier initialization."""
        for layer in self.modules():
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                nn.init.zeros_(layer.bias)

# Training function
# Train surrogate with early stopping
def train_surrogate(x_train_fold, y_train_fold, input_dim, output_dim,
                    num_epochs=500, patience=15, batch_size=64, device='cuda'):
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    x_train_fold, y_train_fold = x_train_fold.to(device), y_train_fold.to(device)

    model = BayesianRegressionNet(input_dim, output_dim).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    def weighted_loss(output, target, weights=[0.5, 0.5]):
        s11_loss = F.smooth_l1_loss(output[:, :100], target[:, :100], beta=0.1)
        gain_loss = F.smooth_l1_loss(output[:, 100:], target[:, 100:], beta=0.1)
        return weights[0] * s11_loss + weights[1] * gain_loss

    dataset = TensorDataset(x_train_fold, y_train_fold)
    train_size = int(0.9 * len(dataset))
    train_set, val_set = random_split(dataset, [train_size, len(dataset) - train_size])
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size * 2, shuffle=False)

    best_loss = float('inf')
    best_weights = None
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = weighted_loss(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                val_loss += weighted_loss(outputs, targets).item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_weights = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    model.load_state_dict(best_weights)
    return model, best_loss

# Configuration
input_dim = 56
output_dim = 200
batch_size = 32
num_epochs = 1000
patience = 20
k = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Convert to torch tensors
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(Y, dtype=torch.float32)
X_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 3-Fold Cross-validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)
val_losses = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"Fold {fold + 1}/{k}")

    x_fold_train = X_train[train_idx]
    y_fold_train = y_train[train_idx]

    model, val_loss = train_surrogate(
        x_fold_train, y_fold_train,
        input_dim=input_dim,
        output_dim=output_dim,
        num_epochs=num_epochs,
        patience=patience,
        batch_size=batch_size,
        device=device
    )

    val_losses.append(val_loss)
    print(f"Fold {fold + 1} Validation Loss: {val_loss:.4f}")

# Final training on full data
print("\nTraining final model on full training set...")
final_model, _ = train_surrogate(
    X_train, y_train,
    input_dim=input_dim,
    output_dim=output_dim,
    num_epochs=num_epochs,
    patience=patience,
    batch_size=batch_size,
    device=device
)

# Test evaluation
final_model.eval()
with torch.no_grad():
    preds = final_model(X_test.to(device))
    s11_loss = F.smooth_l1_loss(preds[:, :100], y_test[:, :100].to(device), beta=0.1)
    gain_loss = F.smooth_l1_loss(preds[:, 100:], y_test[:, 100:].to(device), beta=0.1)
    test_loss = 0.5 * s11_loss.item() + 0.5 * gain_loss.item()
    print(f"\nTest Loss: {test_loss:.4f}")

print(f"Average Cross-Validation Loss: {np.mean(val_losses):.4f}")

Fold 1/3
Fold 1 Validation Loss: 2.4591
Fold 2/3
Fold 2 Validation Loss: 2.8518
Fold 3/3
Fold 3 Validation Loss: 2.8321

Training final model on full training set...

Test Loss: 2.2953
Average Cross-Validation Loss: 2.7143


# Design Space Bounds

In [14]:
def get_design_space_bounds():
    pcb_width = 25  # mm 
    pcb_length = 30  # mm 
    half_width = pcb_width / 2
    half_length = pcb_length / 2

    conductor_ranges = {
        'C1': {'x_start': 0.1, 'x_end_range': (-4, -1), 'y_span': (-half_length, half_length-1)},
#         'C2': {'x_start_range': (-16, -7), 'x_thickness_range': (0.5, 2), 'y_span': (-half_length, half_length-3)},
        'G1': {'x_start': 0.1, 'x_end': -half_width, 'y_start': -half_length, 'y_end_range': (-3, 10)},
#         'G2': {'x_start': -half_width, 'x_thickness_range': (1, 16), 'y_end_range': (2, 8)}
    }

    grid_x_cells = 3 
    grid_y_cells = 4 
    patch_width_range = (2, 4)    # mm 
    patch_length_range = (2, 7)   # mm 
    offset_range = (-2,2)         # mm


    lower_bounds = []
    upper_bounds = []


    lower_bounds = []
    upper_bounds = []

    # Adjust bounds slightly to avoid identical lower and upper bounds
    epsilon = 1e-2

    lower_bounds.extend([
        conductor_ranges['C1']['x_start'] - epsilon, conductor_ranges['C1']['x_end_range'][0], 
        conductor_ranges['C1']['y_span'][0] - epsilon, conductor_ranges['C1']['y_span'][1] - epsilon,

    ])
    upper_bounds.extend([
        conductor_ranges['C1']['x_start'] + epsilon, conductor_ranges['C1']['x_end_range'][1],
        conductor_ranges['C1']['y_span'][0] + epsilon, conductor_ranges['C1']['y_span'][1] + epsilon,

    ])

    for i in range(grid_x_cells * grid_y_cells):
        col = i % grid_x_cells
        row = i // grid_x_cells
        cell_width = half_width / grid_x_cells
        cell_height = pcb_length / grid_y_cells
        center_x = -half_width + cell_width * (col + 0.5)
        center_y = -half_length + cell_height * (row + 0.5)

        lower_bounds.extend([
            max(center_x + offset_range[0] - patch_width_range[1] / 2, -half_width),
            max(center_x + offset_range[0] + patch_width_range[0] / 2, -half_width),
            max(center_y + offset_range[0] - patch_length_range[1] / 2, -half_length),
            max(center_y + offset_range[0] + patch_length_range[0] / 2, -half_length)
        ])
        upper_bounds.extend([
            min(center_x + offset_range[1] - patch_width_range[0] / 2, half_width),
            min(center_x + offset_range[1] + patch_width_range[1] / 2, half_width),
            min(center_y + offset_range[1] - patch_length_range[0] / 2, half_length),
            min(center_y + offset_range[1] + patch_length_range[1] / 2, half_length)
        ])

    g1_y_start = conductor_ranges['G1']['y_start']
    g1_y_end_min = conductor_ranges['G1']['y_end_range'][0]
    g1_y_end_max = conductor_ranges['G1']['y_end_range'][1]

    lower_bounds.extend([
        conductor_ranges['G1']['x_start'] - epsilon, conductor_ranges['G1']['x_end'] - epsilon, 
        g1_y_start - epsilon, g1_y_end_min - epsilon,

    ])
    upper_bounds.extend([
        conductor_ranges['G1']['x_start'] + epsilon, conductor_ranges['G1']['x_end'] +
        epsilon, g1_y_start + epsilon, g1_y_end_max + epsilon,

    ])

    bounds = list(zip(lower_bounds, upper_bounds))
    return bounds

In [15]:
bounds = get_design_space_bounds()
np.shape(bounds)

(56, 2)

In [34]:
def objective_function(S, G):
    
    # Define target values
    target_S11 = -15.0  # We want S11 < -15 dB
    target_gain = 3.0   # We want Gain > 3 dBi

    # Compute S11 penalty (higher than -15 dB is penalized)
    penalty_S11 = torch.clamp(S - target_S11, min=0)  # max(S11 - (-15), 0)
    mean_penalty_S11 = torch.mean(penalty_S11, dim=1)  # Average over frequency

    # Compute Gain penalty (lower than 3 dBi is penalized)
    penalty_gain = torch.clamp(target_gain - G, min=0)  # max(3 - Gain, 0)
    mean_penalty_gain = torch.mean(penalty_gain, dim=1)  # Average over frequency

    # Combine both penalties with equal weight
    total_penalty = 0.5 * mean_penalty_S11 + 0.4 * mean_penalty_gain

    return total_penalty  # Lower fitness is better

In [51]:
def mc_dropout_predict(model, X_test, num_samples=100, batch_size=32):
    model.train()  # Enable MC dropout
    num_test_samples = X_test.size(0)
    device = X_test.device

    mean = torch.zeros((num_test_samples, 200), device="cpu")
    variance = torch.zeros((num_test_samples, 200), device="cpu")

    for start in range(0, num_test_samples, batch_size):
        end = min(start + batch_size, num_test_samples)
        batch = X_test[start:end].to(device)

        batch_predictions = torch.zeros((num_samples, end - start, 200), device="cpu")
        for i in range(num_samples):
            with torch.no_grad():
                batch_predictions[i] = model(batch).detach().cpu()

        mean[start:end] = batch_predictions.mean(dim=0)  # Remove dropout scaling
        variance[start:end] = batch_predictions.var(dim=0) + 1e-6  # Stabilized variance

        del batch, batch_predictions  # Remove redundant torch.cuda.empty_cache()

    return mean.to(device), variance.to(device)


def mc_dropout_predict_samples(model, X_test, num_samples=50, batch_size=32):
    """
    Return raw MC samples (for EI calculation).
    Enables dropout while keeping BatchNorm in eval mode.
    """
    def enable_dropout_eval_only(model):
        for m in model.modules():
            if isinstance(m, torch.nn.Dropout):
                m.train()
            elif isinstance(m, torch.nn.BatchNorm1d):
                m.eval()

    enable_dropout_eval_only(model)

    num_test_samples = X_test.size(0)
    device = X_test.device
    output_dim = 200  # Update this if your model has a different output shape

    # Store all MC samples on CPU (memory-efficient)
    all_samples = torch.zeros((num_samples, num_test_samples, output_dim), device="cpu")

    for start in range(0, num_test_samples, batch_size):
        end = min(start + batch_size, num_test_samples)
        batch = X_test[start:end].to(device)

        # Generate MC samples for the current batch
        batch_samples = []
        for _ in range(num_samples):
            with torch.no_grad():
                pred = model(batch).detach().cpu()
            batch_samples.append(pred)

        batch_samples = torch.stack(batch_samples)  # [num_samples, batch_size, output_dim]
        all_samples[:, start:end, :] = batch_samples

    return all_samples.to(device)


def train_surrogate(X_train, y_train, input_dim=56, output_dim=200, num_epochs=500, patience=30, batch_size=32, device='cuda'):
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    X_train, y_train = X_train.to(device), y_train.to(device)
    
    model = BayesianRegressionNet(input_dim, output_dim).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = nn.MSELoss()
    
    dataset = TensorDataset(X_train, y_train)
    train_size = int(0.9 * len(dataset))
    train_set, val_set = random_split(dataset, [train_size, len(dataset) - train_size])
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size * 2, shuffle=False)
    
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1}")
                break
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1:3d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    return model, best_loss

# EI-DE Algorithm

In [52]:
import torch
from torch.distributions import Normal
from scipy.stats.qmc import LatinHypercube

def lhs_sampling(samples, dims, device="cpu"):
    """Generate Latin Hypercube Samples and convert to PyTorch tensor."""
    lhs = LatinHypercube(d=dims)  # Create LHS sampler for given dimensions
    sample_points = lhs.random(n=samples)  # Generate LHS samples in [0,1]
    return torch.tensor(sample_points, dtype=torch.float32, device=device)

def simplified_DE_optimizer(
    model, f_best, bounds, pcb_width, pcb_length,
    num_samples=50, q=30, maxiter=100,
    popsize=100, F_init=0.9, cr_init=0.9,
    local_search=True, local_search_maxiter=20,
    xi=0.05,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    batch_size=128,
    stagnation_limit=40
):
    def min_obj(X_batch, adaptive_xi):
        if X_batch.dim() == 1:
            X_batch = X_batch.unsqueeze(0)

        mc_samples = mc_dropout_predict_samples(model, X_batch, num_samples=num_samples)
        S = mc_samples[..., :100]
        G = mc_samples[..., 100:]
        obj_values = objective_function(S, G)

        mu = torch.mean(obj_values, dim=0)
        sigma = torch.std(obj_values, dim=0)
        sigma = torch.clamp(sigma, min=1e-3)

        improvement = torch.clamp(f_best - mu - adaptive_xi, min=0)
        Z = improvement / (sigma + 1e-6)
        Z = torch.clamp(Z, min=-5, max=5)
        normal = Normal(0, 1)
        EI = improvement * normal.cdf(Z) + sigma * torch.exp(normal.log_prob(Z))
        EI[sigma < 1e-3] = 0

        return -EI.sum()

    bounds_tensor = torch.tensor(bounds, dtype=torch.float32, device=device)
    dim = bounds_tensor.shape[0]

    population = lhs_sampling(popsize, dim, device=device)
    population = population * (bounds_tensor[:, 1] - bounds_tensor[:, 0]) + bounds_tensor[:, 0]
    population = check_and_adjust_coordinates_within_bounds(population, pcb_width, pcb_length)

    fitness = torch.zeros(popsize, device=device)
    for i in range(0, popsize, batch_size):
        fitness[i:i+batch_size] = min_obj(population[i:i+batch_size], xi)

    best_fitness = fitness.min()
    best_params = population[fitness.argmin()]
    stagnation_counter = 0

    for generation in range(maxiter):
        if stagnation_counter >= stagnation_limit:
            print(f"Early stopping at generation {generation + 1} due to stagnation.")
            break

        # Adaptive F and CR: linearly decreasing/increasing
        F = F_init * (1 - generation / maxiter)
        cr = cr_init * (1 - generation / maxiter)
        adaptive_xi = max(0.1, 0.5 * (stagnation_counter / stagnation_limit))

        idx = torch.randint(0, popsize, (3, popsize), device=device)
        a, b, c = idx[0], idx[1], idx[2]
        mutant = population[a] + F * (population[b] - population[c])
        mutant = torch.clamp(mutant, bounds_tensor[:, 0], bounds_tensor[:, 1])
        mutant = check_and_adjust_coordinates_within_bounds(mutant, pcb_width, pcb_length)

        cross_mask = torch.rand(popsize, dim, device=device) < cr
        trial_pop = torch.where(cross_mask, mutant, population)
        trial_pop = check_and_adjust_coordinates_within_bounds(trial_pop, pcb_width, pcb_length)

        trial_fitness = torch.zeros(popsize, device=device)
        for i in range(0, popsize, batch_size):
            trial_fitness[i:i+batch_size] = min_obj(trial_pop[i:i+batch_size], adaptive_xi)

        improved = trial_fitness < fitness
        population[improved] = trial_pop[improved]
        fitness[improved] = trial_fitness[improved]

        current_best = fitness.min()
        if current_best < best_fitness:
            best_fitness = current_best
            best_params = population[fitness.argmin()]
            stagnation_counter = 0
        else:
            stagnation_counter += 1

        if generation % 10 == 0 or generation == maxiter - 1:
            print(f"Generation {generation + 1}/{maxiter} | Best Fitness: {best_fitness:.6f} | Stagnation: {stagnation_counter}")

    # Local search on top q candidates
    if local_search:
        print(f"Refining top {q} candidates with L-BFGS...")
        top_q_indices = torch.argsort(fitness)[:q]
        selected_points = population[top_q_indices]

        refined_candidates = []
        for idx in range(q):
            x = selected_points[idx].clone().unsqueeze(0).requires_grad_(True)

            optimizer = torch.optim.LBFGS([x], max_iter=local_search_maxiter, line_search_fn='strong_wolfe')

            
            def closure():
                optimizer.zero_grad()
            
                
                enable_dropout_eval_only(model)
            
                loss = min_obj(x, adaptive_xi)
            
                if loss.requires_grad:
                    loss.backward()
            
                return loss

            optimizer.step(closure)

            x.data = torch.clamp(x.data, bounds_tensor[:, 0], bounds_tensor[:, 1])
            x.data = check_and_adjust_coordinates_within_bounds(x, pcb_width, pcb_length)
            refined_candidates.append(x.detach().squeeze(0))

        selected_points = torch.stack(refined_candidates)

    else:
        top_q_indices = torch.argsort(fitness)[:q]
        selected_points = population[top_q_indices]

    return selected_points.cpu().numpy()

#Matlab functions

In [29]:
pcb_width = 25  # mm
pcb_length = 30  # mm
import matlab.engine

def mirror_design_along_y(design, pcb_width, pcb_length):
    """
    Mirrors the design along the y-axis.
    :param design: Torch tensor containing the design of shape [q, 88].
    :param pcb_width: Width of the PCB.
    :param pcb_length: Length of the PCB.
    :return: Torch tensor containing the mirrored design of shape [q, 88].
    """
    mirrored_design = design.clone()
    var_per_component = 4  # x_start, x_end, y_start, y_end
    half_width = pcb_width / 2
    half_length = pcb_length / 2

    for i in range(0, design.shape[1], var_per_component):
        x_start, x_end, y_start, y_end = design[:, i:i + var_per_component].T

        # Mirror the x-coordinates along the y-plane
        mirrored_x_start = -x_end
        mirrored_x_end = -x_start

        # Clamp mirrored x and y coordinates to ensure bounds
        mirrored_x_start = torch.clamp(mirrored_x_start, -half_width, half_width)
        mirrored_x_end = torch.clamp(mirrored_x_end, -half_width, half_width)
        y_start = torch.clamp(y_start, -half_length, half_length)
        y_end = torch.clamp(y_end, -half_length, half_length)

        # Update mirrored design
        mirrored_design[:, i:i + var_per_component] = torch.stack(
            [mirrored_x_start, mirrored_x_end, y_start, y_end], dim=1)

    return mirrored_design

def extract_G1_G2_coords(design):
    """
    Extracts G1 coordinates from the design array.
    Assumes G1 is at indices [-4:].
    """
    return design[:, -4:]  # Directly slice the last 4 columns


def remove_redundant_variables(design):
    """
    Removes G1 coordinates from the design array.
    Assumes G1 is at indices [-4:].
    """
    return design[:, :-4]


def matlab_simulation_function(design):
    """
    Calls the MATLAB simulation to predict VSWR for the given PCB design.
    :param design: Torch tensor containing the design.
    :return: Torch tensor with the simulated VSWR values.
    """
    # Save the design to a text file, converting to CPU if necessary
    design_numpy = design.cpu().numpy()
    np.savetxt('combined_pcb_new.txt', design_numpy, delimiter='\t')

    # Start MATLAB engine and run the simulation script
#     eng = matlab.engine.start_matlab()
    eng.Ant_sim56(nargout=0)

    # Load the VSWR result from the file generated by MATLAB
    S11_result = np.loadtxt('S11_t.txt', delimiter='\t')
    gain_result = np.loadtxt('Gain_t.txt', delimiter='\t')
    
    combined_results = np.concatenate((S11_result, gain_result), axis=1)

    # Convert to torch tensor and return (on same device as input)
    return torch.tensor(combined_results, dtype=torch.float32, device=design.device)

def check_and_adjust_coordinates_within_bounds(combined_pcb_variables, pcb_width, pcb_length):
    """
    Ensures all coordinates in the design are within the PCB bounds.
    :param combined_pcb_variables: Torch tensor containing the combined design variables.
    :param pcb_width: Width of the PCB.
    :param pcb_length: Length of the PCB.
    :return: Torch tensor with adjusted coordinates.
    """
    half_width = torch.tensor(pcb_width / 2, dtype=combined_pcb_variables.dtype, device=combined_pcb_variables.device)
    half_length = torch.tensor(pcb_length / 2, dtype=combined_pcb_variables.dtype, device=combined_pcb_variables.device)

    adjusted_coords = combined_pcb_variables.clone()

    for i in range(0, adjusted_coords.shape[1], 4):
        # Extract and adjust x and y coordinates
        x_start, x_end, y_start, y_end = adjusted_coords[:, i:i+4].T

        x_start = torch.clamp(x_start, -half_width, half_width)
        x_end = torch.clamp(x_end, -half_width, half_width)
        y_start = torch.clamp(y_start, -half_length, half_length)
        y_end = torch.clamp(y_end, -half_length, half_length)

        # Update adjusted coordinates
        adjusted_coords[:, i:i+4] = torch.stack([x_start, x_end, y_start, y_end], dim=1)

    return adjusted_coords


# Bayesian Optimization Loop

In [53]:
def bayesian_optimization(X_train, y_train, bounds, num_iterations=100, xi=0.05, num_samples=100, q=30):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    bounds = np.array(bounds)
    input_dim = bounds.shape[0]

    X_train = X_train.to(device)
    y_train = y_train.to(device)

    # Load or train model (handled by train_surrogate)
    model, _ = train_surrogate(X_train, y_train, input_dim=input_dim,output_dim=output_dim, device=device)
    
    best_design = None
    best_objective_value = float('inf')
    no_improvement_count = 0
    # Open files for saving best values
    best_objective_file = open("best_objective_per_iteration.txt", "a")
    best_design_file = open("best_design_per_iteration.txt", "a")
    for iteration in range(num_iterations):
        if no_improvement_count >= 15:
            print(f"Early stopping at iteration {iteration + 1}")
            break

        # Calculate f_best from ALL historical data
        all_scores = objective_function(y_train[:, :100], y_train[:, 100:])
        f_best = all_scores.min()
        print('Current f_best:', f_best.item())

        # Get next samples using BEST model weights
        next_samples = simplified_DE_optimizer(model, f_best.item(), bounds, popsize =popsize,pcb_width = pcb_width, pcb_length=pcb_length, 
                                             num_samples=num_samples, q=q, device=device)
        next_samples_tensor = torch.tensor(next_samples, dtype=torch.float32, device=device)

        combined_pcb_variables_list = []
        for next_sample in next_samples_tensor:
            mirrored_sample = mirror_design_along_y(next_sample.unsqueeze(0), pcb_width, pcb_length).to(device)
            G1_original = extract_G1_G2_coords(next_sample.unsqueeze(0)).to(device)
            G1_mirrored = extract_G1_G2_coords(mirrored_sample).to(device)
            next_sample_trimmed = remove_redundant_variables(next_sample.unsqueeze(0)).to(device)
            mirrored_sample_trimmed = remove_redundant_variables(mirrored_sample).to(device)
            combined_pcb_variables = torch.cat((next_sample_trimmed, mirrored_sample_trimmed, G1_original, G1_mirrored), dim=1).to(device)
            combined_pcb_variables = check_and_adjust_coordinates_within_bounds(
                combined_pcb_variables, pcb_width, pcb_length
            ).to(device)
            combined_pcb_variables_list.append(combined_pcb_variables)
        
        all_combined_pcb_variables = torch.cat(combined_pcb_variables_list, dim=0).to(device)
        next_samples_output = matlab_simulation_function(all_combined_pcb_variables).to(device).view(q, -1)

        X_train = torch.cat((X_train, next_samples_tensor), dim=0).to(device)
        y_train = torch.cat((y_train, next_samples_output), dim=0).to(device)

        model, _ = train_surrogate(X_train, y_train, input_dim=input_dim, device=device)
        

        current_scores = objective_function(next_samples_output[:, :100], next_samples_output[:, 100:])
        current_best_output = torch.tensor(current_scores, device=device).min().item()
        # Save best objective and best design per iteration
        best_objective_file.write(f"{current_best_output:.6f}\n")
        np.savetxt(best_design_file, next_samples_tensor.cpu().numpy()[:1], delimiter='\t')

        if current_best_output < best_objective_value:
            best_objective_value = current_best_output
            np.savetxt('best_design.txt', next_samples_tensor.cpu().numpy(), delimiter='\t')
            np.savetxt('best_design_output.txt', next_samples_output.cpu().numpy(), delimiter='\t')
        else:
            no_improvement_count += 1
        
        print(f'Iteration {iteration + 1}/{num_iterations} completed. Best objective value: {best_objective_value:.4f}')
    best_objective_file.close()
    best_design_file.close()
    return model, X_train, y_train, best_design


In [55]:
import datetime
t1 = datetime.datetime.now()
print(t1)
bounds = get_design_space_bounds()
maxiter = 100  
popsize = 100
input_dim=56
output_dim = 200
num_iterations = 100  
q = 30
num_samples = 100
# model,_ = train_surrogate(X_train, y_train, input_dim=56, output_dim=200,num_epochs=500, patience=15, 
#                           batch_size=64, device=device)
eng = matlab.engine.start_matlab()
model, X_train, y_train, best_design  = bayesian_optimization(X_train, y_train, bounds, num_iterations=num_iterations, xi=0.05, num_samples=num_samples, q=q)
print('X_train Shape:',X_train.shape,"Best Design:", best_design)
t2 = datetime.datetime.now() - t1
print('time_taken=',t2)
eng.quit()